In [5]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import os
import sys

import plotly.graph_objects as go
import plotly.express as px
import pandas as pd


import pandas as pd
import plotly.express as px
import plotly.graph_objs as go

sys.path.insert(0, os.path.abspath('../developer'))

from config import MOCK_DATA, CODE, OUT, IN
from developer.utilities import read_yaml

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Frequencies of attributes

In [4]:
conjoint_reg = pd.read_csv(OUT / "data" / "data_regression.csv")

#conjoint_dummy.columns.str.startswith('att')
def frequencies(conjoint_reg):
    
    frequency_table =  {}
    groups = ['att_1', 'att_2','att_3', 'att_4', 'att_5'] #add 6
    for group in groups:
        total = sum(dict(conjoint_reg.filter(like=group).sum()).values())
        frequency_table[group] = {key.replace(f'{group}_', '') : (value / total).round(2) for key, value in dict(conjoint_reg.filter(like=group).sum()).items()}
    frequency_table = pd.DataFrame(frequency_table).sum(axis=1)
    frequency_table = pd.DataFrame(frequency_table, columns=["frequency"])

    frequency_table = frequency_table.rename_axis("Attribute_level")
    return frequency_table

frequencies(conjoint_reg)

,frequency
Attribute_level,
Eliminate&UseAllOther,0.25
Eliminate&UseRenewables,0.23
Reduce&IncreaseAllOther,0.27
Reduce&IncreaseRenewables,0.25
EnergyAccess,0.26
InsureWorkers,0.22
LowPrices,0.28
NothingSoc,0.25
IdentityCoalRegions,0.26


#### Time spent on surveys

In [49]:

import plotly.express as px
import plotly.figure_factory as ff


clean_data = pd.read_csv(OUT / "data" / "data_clean.csv")

clean_data['DurationMin'] = clean_data['Duration (in seconds)'] / 60

clean_data = clean_data[clean_data['potentially_unengaged'] == 0]

# Calculate the average and median
average_duration = clean_data['DurationMin'].mean()
median_duration = clean_data['DurationMin'].median()

fig = ff.create_distplot([clean_data['DurationMin']], ['DurationMin'], show_curve=True, show_rug=False, bin_size=2.0)

fig.add_shape(
    type='line',
    x0=average_duration,
    x1=average_duration,
    y0=0,
    y1=1,
    xref='x',
    yref='paper',
    line=dict(color='red', width=2, dash='dash'),
    name=f'Average: {average_duration:.2f}'
)

fig.add_shape(
    type='line',
    x0=median_duration,
    x1=median_duration,
    y0=0,
    y1=1,
    xref='x',
    yref='paper',
    line=dict(color='green', width=2, dash='dash'),
    name=f'Median: {median_duration:.2f}'
)

# Add labels for average and median values
fig.add_annotation(
    x=average_duration,
    y=1.08,
    xref='x',
    yref='paper',
    text=f'Average: {average_duration:.2f}',
    showarrow=False,
    arrowhead=4,
    ax=0,
    ay=-40
)

fig.add_annotation(
    x=median_duration,
    y=1.13,
    xref='x',
    yref='paper',
    text=f'Median: {median_duration:.2f}',
    showarrow=False,
    arrowhead=4,
    ax=0,
    ay=-40
)

# Update the layout if needed
fig.update_layout(
    title='Density Plot with Smooth Line',
    xaxis_title='Value',
    yaxis_title='Density'
)
# Update the layout if needed
fig.update_layout(title='Fig B: Density plot of time spent for engaged', xaxis_title='Value', yaxis_title='Density')


# Show the plot
fig.show()

#### Treatment

In [43]:
import pandas as pd

categories = ['Eliminate&UseAllOther', 'Eliminate&UseRenewables', 'Reduce&IncreaseAllOther', 'Reduce&IncreaseRenewables']
categories = ['LowPrices', 'InsureWorkers', 'EnergyAccess', 'NothingSoc']
categories = ['IdentityCoalRegions', 'IndustryCoalRegions', 'WorkersCoalRegion', 'NothingEco']
categories = ['Gov&CivilSociety', 'Gov&Researchers', 'Gov&Businesses', 'Gov&LaborUnions', 'Gov&LocalGov', 'GovAlone']
categories = ['Fin&TechSupport', 'FinSupportOnly', 'TechSupportOnly', 'NoInterference']

def attribute_support(df, attribute, categories):
    df = df.copy()

    df_control = df[df['treatment_status'] == 0]
    df_treated = df[df['treatment_status'] == 1]

    df_treated = df_treated[[attribute, 'support']]
    df_treated['support'] = df_treated['support'].astype(int)

    support_control = {"Attribute Level": [], "Value": [], "CI_lower": [], "CI_upper": []}
    
    for cat in categories[::-1]:
        group = df_control[df_control[attribute] == cat]
        mean = group['support'].mean()
        std_dev = group['support'].std()
        n = len(group)
        confidence_interval = 1.96 * (std_dev / (n**0.5))  # 95% confidence interval

        support_control["Attribute Level"].append(cat.replace('&', '<br>'))
        support_control["Value"].append(mean.round(2))
        support_control["CI_lower"].append((mean - confidence_interval).round(2))
        support_control["CI_upper"].append((mean + confidence_interval).round(2))

    df_control = pd.DataFrame(support_control)

    support_treated = {"Attribute Level": [], "Value": [], "CI_lower": [], "CI_upper": []}
    
    for cat in categories[::-1]:
        group = df_treated[df_treated[attribute] == cat]
        mean = group['support'].mean()
        std_dev = group['support'].std()
        n = len(group)
        confidence_interval = 1.96 * (std_dev / (n**0.5))  # 95% confidence interval

        support_treated["Attribute Level"].append(cat.replace('&', '<br>'))
        support_treated["Value"].append(mean.round(2))
        support_treated["CI_lower"].append((mean - confidence_interval).round(2))
        support_treated["CI_upper"].append((mean + confidence_interval).round(2))

    df_treated = pd.DataFrame(support_treated)

    color_scale = ["rgb(173, 221, 142)", "rgb(127, 188, 65)", "rgb(78, 139, 37)", "rgb(45, 82, 21)", "rgb(45, 82, 21)", "rgb(45, 82, 21)","rgb(45, 82, 21)"]

    fig = go.Figure()

    for i, row in df_control.iterrows():
        fig.add_trace(go.Bar(
            x=[row["Attribute Level"]],
            y=[row["Value"]],
            error_y=dict(
                type='data',
                array=[row["CI_upper"] - row["Value"]],
                arrayminus=[row["Value"] - row["CI_lower"]],
                visible=True
            ),
            marker_color=color_scale[i],
            name=row["Attribute Level"],
            width=0.2
        ))

    color_scale = ["rgb(221, 142, 142)", "rgb(188, 65, 65)", "rgb(139, 37, 37)", "rgb(82, 21, 21)", "rgb(82, 21, 21)", "rgb(82, 21, 21)", "rgb(82, 21, 21)"]


    for i, row in df_treated.iterrows():
        fig.add_trace(go.Bar(
            x=[row["Attribute Level"]],
            y=[row["Value"]],
            error_y=dict(
                type='data',
                array=[row["CI_upper"] - row["Value"]],
                arrayminus=[row["Value"] - row["CI_lower"]],
                visible=True
            ),
            marker_color=color_scale[i],
            name=row["Attribute Level"],
            width=0.2
        ))

    

    # Set y-axis range from 0 to 1
    fig.update_layout(yaxis_range=[0, 1], xaxis_range=[-1,4], width=700, height=500)

    # Add a horizontal line at y=0.5
    fig.add_hline(y=0.5, line_dash="dash")


    fig.update_layout(barmode="group", bargap=0.6)
    
    fig.update_layout(
        title={
            'text': "Fig 5: Support of the different international support",
            'x': 0.5,
            'xanchor': 'center',
            'font': {'family': 'Computer Modern'}
        },
        margin=dict(l=20, r=20, t=45, b=5),
        paper_bgcolor="#EADDCA",
        plot_bgcolor='rgba(0,0,0,0)',
        showlegend=False,  # Show legend for different Attribute Levels
        xaxis_showticklabels=True,
        xaxis_title=None,
    )

    return fig

data_clean = pd.read_csv(OUT / "data" / "data_long.csv")

attribute_support(data_clean, "att_5", categories)